In [1]:
#Librerias backend:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Librerias frontend:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, Flatten, MaxPooling2D
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Presentacion del dataset y objetivo de la convnet:

Se tiene un dataset contiendo imagenes de 16 piezas de lego distintas. El objetivo es lograr predecir al nombre de la pieza mostrada en la imagen.

In [2]:
#plt.imshow("/content/drive/My Drive/MLII/CNN/LEGO brick images/train/11214 Bush 3M friction with Cross axle/201706171006-0001.png")

Para la implementacion del modelo se utilizará Keras, dada la simplicidad de implementacion y la flexibilidad en cuanto a la eleccion de la arquitectura.

# Architectura de la CNN:

Se tomaran en cuenta las recomendaciones hechas en clase:

* Se utilizara la funcion de activacion ReLu.
* Se Utilizará Max Pooling (lo que hay "mas importante")
* No se utiizará **dropaut**
* Dado que el dataset consiste en imagenes renderizadas de piezas de lego rotadas, se evaluará la opcion de realizar aumentado de datos.
* Se utilizará el optimizador Adam 
* Se utilizará un batch size de 64

Las imagenes tiene un tamaño estandard de 72x72, siendo cada una de 3 canalaes. Por lo tanto, mi capa de entrada será de: 72x72x3.

La capa convolucional

# Carga del dataset y preprocesamiento de las imagenes.

Fuente: https://keras.io/preprocessing/image/


In [3]:
imageWidth = 72
imageHeight = 72 
imageChannel = 3
testPorcen = 0.15
batchSize = 64
dataPath = 'train'

Declaracion del generador para preprocesar las imagenes. Se tomara un 15% de datos para test:

In [4]:
#Se utilizaran los valores por defecto de la generador:
train_datagen = ImageDataGenerator(
        validation_split=testPorcen, #Split argument for data division.
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

Procedemos con la separacion de la data en Train y Test. **Debido a que especificamos el valor de validation_split en el generador anterior, ahora debemos especificar que datos son de validacion (test) y cuales son de train.**:

In [5]:
train_generator = train_datagen.flow_from_directory(
        dataPath,
        target_size=(imageHeight, imageWidth),
        batch_size=batchSize,
        class_mode='categorical', #Por defecto es categorical, sin embargo, no esta de más settearla manualmente.
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        dataPath,
        target_size=(imageHeight, imageWidth),
        batch_size=batchSize,
        class_mode='categorical', #Por defecto es categorical, sin embargo, no esta de más settearla manualmente.
        subset='validation')

Found 5423 images belonging to 16 classes.
Found 956 images belonging to 16 classes.


Procedemos a aplicar los generadores para obtener nuestra deta:

# Creacion del modelo:

Fuente: https://towardsdatascience.com/boost-your-cnn-image-classifier-performance-with-progressive-resizing-in-keras-a7d96da06e20

In [6]:
#Valores de la capa convolucional:
filtersNumber = 8
filterSize = 5
pool_size = 3
fully_connected_neurons = 64
drop_out_factor = 0.6
classes_out = 16 #numero de tipos de legos (neuronas de salida.)
numEpochs = 10

In [7]:
model = Sequential()
#Primera capa de convolución (se estan tomando en cuenta todas las recomendaciones indicadas: Funcion de activacion Relu, Mini Batch de 64)
model.add(Conv2D(batchSize, input_shape=(imageHeight, imageWidth, imageChannel) , activation='relu', kernel_size=(filterSize, filterSize)  ))
#Pooling, tomando en cuenta la recomendacion de utilizar MaxPooling:
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


#Segunda capa de convolucion:
model.add(Conv2D(batchSize, input_shape=(imageHeight, imageWidth, imageChannel) , activation='relu', kernel_size=(filterSize, filterSize)  ))
#Pooling para la segunda capa:
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


#Aplanado de la ultima capa de pooling con las respectivas caracteristicas para la parte "fully connected":
model.add(Flatten())
#Se utilizaran 64 neuronas en la etapa fully connected
model.add(Dense(fully_connected_neurons, activation='relu'))
#En esta etapa si se utilizará dropout, ya que equivale a un modelo MLP:
model.add(Dropout(rate=drop_out_factor))
#Se tienen 16 neuronas de salida, 1 para cada posible valor de la salida. Se utiliza Softmax puesto que necesitamos una probabiliad
model.add(Dense(classes_out, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 68, 68, 64)        4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________

In [8]:
#Compilando el modelo:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
#Entrenando el modelo:
preddicted = model.fit_generator(
    train_generator,
    use_multiprocessing=True,
    steps_per_epoch=len(train_generator.filenames) // batchSize,
    epochs=numEpochs,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batchSize
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
84/84 [==============================] - 55s 659ms/step - loss: 2.3547 - acc: 0.2089 - val_loss: 1.8632 - val_acc: 0.3463
Epoch 2/10
84/84 [==============================] - 54s 649ms/step - loss: 1.9664 - acc: 0.3115 - val_loss: 1.6603 - val_acc: 0.3838
Epoch 3/10
84/84 [==============================] - 55s 650ms/step - loss: 1.7549 - acc: 0.3843 - val_loss: 1.5033 - val_acc: 0.4199
Epoch 4/10
84/84 [==============================] - 59s 698ms/step - loss: 1.6247 - acc: 0.4165 - val_loss: 1.3267 - val_acc: 0.4963
Epoch 5/10
84/84 [==============================] - 58s 687ms/step - loss: 1.5124 - acc: 0.4569 - val_loss: 1.3174 - val_acc: 0.5351
Epoch 6/10
84/84 [==============================] - 56s 665ms/step - loss: 1.4172 - acc: 0.4784 - val_loss: 1.2662 - val_acc: 0.5289
Epoch 7/10
84/84 [==============================] - 57s 683ms/step - loss: 1.3250 - acc: 0.5042 - val_loss: 1.1684 - val_acc: 0.5521
Epoch 8/10
84/84 [===

In [15]:
listBatch = []
for i in range(6):
    xTestBatch, yTestBatch = validation_generator.next()
    accuracy = np.mean(1.0*(model.predict_classes(xTestBatch) == np.argmax(yTestBatch,axis=1)))
    listBatch.append(accuracy)
    print("Batch sample: "+ str(i)+" accuracy: "+ str(accuracy))
print(np.mean(listBatch))

Batch sample: 0 accuracy: 0.640625
Batch sample: 1 accuracy: 0.578125
Batch sample: 2 accuracy: 0.5625
Batch sample: 3 accuracy: 0.65625
Batch sample: 4 accuracy: 0.625
Batch sample: 5 accuracy: 0.5625
0.6041666666666666


# Se obtiene un accuracy del 60% en promedio, por lo que se modificara el modelo.

Hiperparametros a resaltar:
* Batch size de 64
* Tamaño de los filtros en las capas convolucionales: 5x5
* Epochs: 10
* Optimizador: Adam